# [Quantum Machine learning in NISQ era](https://my.rolos.com/public/project/qml_algs_review)
<img src="https://www.microsoft.com/en-us/research/uploads/prod/2022/03/Quantum-blog_ChetanNayak_03-2022_1400x788.jpg" height=250>
<img src="https://pennylane.ai/images/qml/whatisqml/QML_optimization.svg" height=250>
*

In this repo presented several algorithms for quantum computers that have a hope for achieving advantage over classical models at least in some tasks in Noisy Intermediate Scale Quantum (NISQ) era or in nearest future.

Review is devided into these main parts:

1. **QML frameworks**  
  Firstly tools for running quantum algorithms by simulation or by real quantum computers. Frameworks' abilities and limitations are presented.
1. **Continuous Variable Quantum Computing**  
  Foundations of photonic approach to quantum computing with infinite dimensional qumodes instead of 2-dim qubits. 
2. **Generative models**
   - **Quantum Recurrent Unit on Gaussian platform**  
   Continuous variable based algorithm used for text translation.
   - **Quantum GAN-s**  
   Algorithms with widely believed exponential advantage over classical counterparts.   
3. **Category theory and ZX-calculus**     
    Mathematically justified diagrammatic language for writing quantum circuits
4. **Quantum Graph classification models**  
Exponential advantage due to parallel processing of subgraphs. And natural quantum graph embedding based on number of perfect matchings in subgraphs. 

\* images from [1](https://www.microsoft.com/en-us/research/uploads/prod/2022/03/Quantum-blog_ChetanNayak_03-2022_1400x788.jpg) and [2](https://pennylane.ai/images/qml/whatisqml/QML_optimization.svg)

All these topics discussions with our code, where needed, are located in the corresponding folders inside rolos project **https://my.rolos.com/public/project/qml_algs_review** . Follow the link to copy project.

Below you can see only `Generative models` part as a preview.

# Generative models

## Quantum Recurrent Unit on Gaussian platform

Contextual Recurrent Network was proposed in (Anschuetz et al. 2023) as an evolution and development of ideas from (Arute et al. 2019). Arute and others considered k-gram quantum models. They showed, that quantum based modification of Bayesian network performs better, than it's classical counterpart.

<!-- ![Performance plots](https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/QBayCorrResults.png)\ -->
<img src=https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/QBayCorrResults.png width=500>\
Comparison between classical and quantum model from (Arute et al. 2019).

This improvement was explained with quantum correlation, which are hard to capture using classical models.
Authors also provide a proof of expressive power separation in absolute perfect case. They demonstrate, that classical model with the same parameters won't be able to achieve finite KL divergence, while quantum, due to quantum nonlocality and contextuality, at least in theory is able.

Anschuetz et al generalized this proof for neural networks. Quantum model, that they proposed is continuous variable(CV) based. So it is designed to use photonic quantum computer, there instead of every qubit one quantum harmonic oscillator is used.
### QRNN architecture
For simulation and general simplicity author limit model to only gaussian states. State of CV device can be represented by very difficult nongaussian Wigner quasiprobability function. But initial vacuum state is just a gaussian with zero mean and unit dispersion. So if only gaussian operations are used (operations that transform gaussian to gaussian. e.g. shifting or squeezing), state will keep being gaussian. 

So if device has $n$ qumods, it's state is fully characterized by vector of means of length $n$ and $n\times n$ symmetric matrix of covariances. In experiments simulation, that stores these numbers is used.
Then evolution of system is described not by specific set of quantum operators (gates), but by general gaussian unitary operation. In fact just square matrix $W$ with size $n+m$ (where n -- hidden state size, and m -- output size). It is the most general gaussian operation, that can be applied to such system. And this matrix is learned with gradient based technic.

<img src=https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/qrnn_visual.png width=500>\
Authors visualization of model (Anschuetz et al. 2023)

To feed the input data to the model, additional $m$ qumods are allocate in quantum device. Input is translated into covariance matrix and means of $m$ qumods via classical fully connected layers. $n$ memory qumod states are also additionally shifted with result of applying another fully connected layer to the input.
Finally the hole $n+m$ size system is transformed with $(n+m) \times (n+m)$ matrix $W$. This mixed memorized information with information obtained on current step.

To get output from this model, expectation values on temporal $m$ qumods are measured.


As a result QRNN performs in natural language translation task slightly better, than classical models in terms of final KL divergence.\
<img src=https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/qrnn_res.png?0 width=500>

Independent implementation of QRNN, is presented in `QRNN` folder. It shows results similar to ones reported by the authors.

Our results are available in wandb platform https://wandb.ai/amirfvb/CRNN_translate

### Reference
Anschuetz, Eric R., Hong-Ye Hu, Jin-Long Huang, and Xun Gao. 2023. “Interpretable Quantum Advantage in Neural Sequence Learning.” PRX Quantum 4 (2): 020338. https://doi.org/10.1103/PRXQuantum.4.020338.

Arute, Frank, Kunal Arya, Ryan Babbush, Dave Bacon, Joseph C. Bardin, Rami Barends, Rupak Biswas, et al. 2019. “Quantum Supremacy Using a Programmable Superconducting Processor.” Nature 574 (7779): 505–10. https://doi.org/10.1038/s41586-019-1666-5.


## Quantum GAN-s

The starting point of using quantum models for generative tasks was done by two articles published together (Dallaire-Demers and Killoran 2018) more practical and (Lloyd and Weedbrook 2018) more theoretical one.

Authors claimed that quantum models may have an exponential advantage over classical models.

This advantage comes from the fact, that measuring quantum system produces a random sample from exponentially big Hilbert Space.

These have more theoretical importance than practical, because sampling itself is rarely used in quantum machine learning. Most approaches (see for example [pennylane demos](https://pennylane.ai/qml/demonstrations/quantum-machine-learning) ) utilize expectation values of output over each qubit. On the one hand expectation value approach doesn't fully explore the potential of quantum computer. For example it fully ignores quantum correlations over different qubits. But on the other hand this approach is more practical. 

To estimate probabilities of each outcome on real quantum computer, you need exponentially many samples (to keep statistical error $1/\sqrt{n}$ constant with exponentially increasing number of outcomes). At the same time this makes expectation value more noise resistent, which is vital property in NISQ era. Outcomes themselves are only finite set, while in many practical tasks continuous model prediction (like expectation value) is required.
And another significant benefit of expectation value is differentiability. There is a parameter shift technic that replaces infinitely small difference in gradient definition with constant difference. So parameter shift technic makes gradient estimation base on medium number of samples averaging possible. 

Despite these difficulties, there are several attempts to demonstrate any quantum advantage with generative models on practical tasks.

### Molecules generation

One of the attempts is done in QuMolGAN paper (Li, Topaloglu, and Ghosh 2021). Authors took classical GAN model for generating molecules MolGAN (N. De Cao, De Cao, and Kipf 2018), and inserted small (2-4 qubit) quantum circuit in the beginning of the generator. After the quantum circuit goes normal classical layers. That's why authors call their model Quantum GAN with hybrid generator.

Authors conclude, that their quantum model is more efficient, than classical one. And quantum model can be reduced to only 15% of parameters to show performance comparable with results of nonreduced classical model. 

During training stage Fréchet distance was measured (see figure below)

<img src=https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/qumolgan_a_plots.png width=500>\
authors plot with learning of quantum and classical GANs.

By dint of code, provided by author, we reproduced model learning.

Unfortunately, proper results visualization shows, that there is no significant advantage of quantum model over comparable classical counterpart.

In terms of Fréchet distance Quantum Mean Reduced (MR) model shows even worse performance, than Classical MR model, which can be seen on smoothed plots below

<img src=https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/qumolgan_my_plots.png?0 width=500>

Moreover, plot of validity score (fraction of valid molecules from generated) shows, that for all models after certain moment there are only vanishingly few number of valid molecules.

<img src=https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/qumolgan_my_plots_validity.png width=500>

In QuMolGAN quantum part was used as a parameterized quantum circuit (PQC), which means, that expectation values were used as a quantum circuit output. In such configuration output is fully determined by parameters of PQC (angles of qubit rotations). To get any randomness, required for generative model, part of the parameters were replaced with random numbers. In this sense, model work pretty like classical GANs, that just transform input distribution to some distribution similar to target one.

Another worth mentioning technic, used in this paper is patching of quantum circuit. 

To generate random vector of specific size with quantum circuit, this vector is divided into $k$ equal parts. And each part is generated separately with small quantum circuits. This approach gives less expressive, as it is a strong restriction to possible circuits width, and size of used Hilbert space is rather small. But it can be necessary in circumstances of NISQ era with quantum devices strongly limited in number of physical qubits, and with classical simulators limited with number of achievable qubits to simulate. With constant device requirement this technic increases dimensionality of output to values closer to practically applicable. 

With molecule generation this idea hasn't shown any advantage, but it was successfully applied in earlier work (Huang et al. 2021).

### Entire Image Generation

<img src="https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/patched_gan.png" width=300>\
authors visualization of their model

Patch technic was applied to generate raw images of digits without any classical postprocessing. Even with patch simplification, authors had to downscale images to 8x8 pixels.

<img src="https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/101.png" width=900>\
Authors results of generating 0 and 1 digits.

Authors use very simple quantum circuit, and they supply classical random noise to get random output. But remarkably they use probabilities of each outcome as an output. Not expectation values, like in many other works. 5 qubits was used, one of which is ancillary, and 4 output qubits. This gives $2^4=16$ output values. Ancillary qubit gives additional nonlinearity for measured output. As mentioned above, this increases required number of measurements (authors used 3000 samples). But it exponentially increases dimensionality of output data and better unleashes potential of quantum computer.

### Compressed images generation

In QuGAN paper (Stein et al. 2021) authors compressed digits images to low dimensional (2-4 dimensions) space by principal component analysis (PCA). This makes the task extremely simple, but at the same time more controllable and convenient for experiments.


<img src="https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/real.png" height=80>\
<img src="https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/1.png" height=80>\
<img src="https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/2.png" height=80>\
<img src="https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/3.png" height=80>\
<img src="https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/4.png" height=80>\
<img src="https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/6.png" height=80>\
<img src="https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/10.png" height=80>\
Demonstration of PCA compression + decompression with different numbers of dimension.

<img src="https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/pca_dist.png" height=300>\
Distribution of PCA compressed data points (first 2 dimensions)

On the picture above you can see visualization of all 10 digits, but only three digits (3,6,9) where used in experiments.

#### Model architecture
In some sense this model is the most quantum among discussed here. It doesn't use classical random noise as an input, but uses quantum random. But they use quantum random in quite specific way.
For $n$ dimensional compression $n$ qubit circuit is used. Instead of estimating expectation value over qubits by averaging hight number(~1000) of samples, they average medium (20-30) number of samples. So to certain extend this produces expectation value with significant statistical uncertainty. This results in some distribution. In corner case of 1 sample procedure will result in discrete probability distribution over single measurement outcome ($2^n$ series of "0" and "1"). Corner case of averaging over $\inf$ samples results in one real number from [0, 1] over each of $n$ dimensions. 
Something in between with $m$ samples will result in discrete distribution(binomial distribution) over $m+1$ equally spaced values over each dimension. As it is an averaging of the samples from constant distribution, according to central limit theorem it approaches normal distribution quite fast. 
This means, that such models won't be able to approximate any complicated distribution far from unimodel "normal like" distribution. And furthermore width of distribution (mathematically speaking "dispersion") is mostly determined by number of samples. And number as far as it is integer it can't be learned efficiently with gradient decent. 

Discriminator architecture is also worth mentioning. Discriminator is quantum and represented by another PQC. Discriminator produces $n$ qubits state vector and accesses generator prediction via dot product in Hilbert space. Dot product value is used as a loss function of discriminator and generator. 
Real data samples are fed to discriminator in the same way after encoding to quantum state. They are encoded by simple single qubit rotations so that expectation values over qubits are equal to true values of data sample.   

#### Experiments

<img src="https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/pca_author.png" height=400>\
author's results visualization

Authors report significant advantage of quantum model over classical GAN in terms of both number of parameters and visual correctness (see figure above)
Source code for quantum model was provided, but due to it's low quality, some error and controversial choice of quantum framework it was reproduced, fixed and complemented by us. Pennylane framework was used instead of qiskit.

Code is available in `QuGAN` folder

It shows, that with right hyperparameter adjust 20 parameter classical generator can approximate distribution much better, than authors of the article showed.

||Classical|Hybrid (Q-Gen, C-Disc)|
|---|---|---|
|image|<img src="https://raw.githubusercontent.com/Hacker1337/QML_review/f27334cc67f0675854cfce04df67f6facd5b95de/img/dft_contourf_530_c_20_params_id=25.png?0" height=150>|<img src="https://raw.githubusercontent.com/Hacker1337/QML_review/f316ee2f2ab15db3c29e327e55b20bde75d3dae0/img/dft_contourf_h_6_params_id=65.png" height=150>|
|hellinger_dist|0.7|0.24|
|# Gen. params| 20| 6 |

different generative models prediction results

Although hybrid model finally has smaller hellinger distance, it can learn only simple distributions similar to normal distribution. And as width of distribution can't be learned by model and is determined by the number of averaged samples, it requires search of this hyperparameter optimal value.

Fully quantum model(QuGAN), presented in (Stein et al. 2021) suffers from several problems with discriminator. 

<img src="https://raw.githubusercontent.com/Hacker1337/QML_review/f316ee2f2ab15db3c29e327e55b20bde75d3dae0/img/QuGAN_discrim.png" width=300>\
author's visualization of quantum discrimination

Firstly, as quantum discriminator accesses directly quantum state, produced by generator, it has no information about sampling, that is used afterwords. In other words discriminator sees only mean value of generated distribution.

Secondly, which is partially saves from previous problem, but still not really good, discriminator output in this architecture is just scalar product between generator output and discriminator state. So loss profile can't be difficult or even can't converge to uniform distribution (see figure below).

<img src="https://raw.githubusercontent.com/Hacker1337/QML_review/f316ee2f2ab15db3c29e327e55b20bde75d3dae0/img/media_images_disciminator_predictions_500_fd43005df664f4b57759.png" width=300> <br>
discriminator predictions. Lighter == higher probability of being fake


All results are available in wandb platform https://wandb.ai/amirfvb/QuGAN-mnist


### Reference:
Dallaire-Demers, Pierre-Luc, and Nathan Killoran. 2018. “Quantum Generative Adversarial Networks.” Physical Review A 98 (1): 012324. https://doi.org/10.1103/PhysRevA.98.012324.

Lloyd, Seth, and Christian Weedbrook. 2018. “Quantum Generative Adversarial Learning.” Physical Review Letters 121 (4): 040502–040502. https://doi.org/10.1103/physrevlett.121.040502.

Li, Junde, Rasit Topaloglu, and Swaroop Ghosh. 2021. “(QuMolGAN) Quantum Generative Models for Small Molecule Drug Discovery.” arXiv. http://arxiv.org/abs/2101.03438.

N. De Cao, Nicola De Cao, and Thomas Kipf. 2018. “MolGAN: An Implicit Generative Model for Small Molecular Graphs.” ArXiv: Machine Learning, May. https://arxiv.org/abs/1805.11973.

Huang, He-Liang, Yuxuan Du, Ming Gong, Youwei Zhao, Yulin Wu, Chaoyue Wang, Shaowei Li, et al. 2021. “Experimental Quantum Generative Adversarial Networks for Image Generation.” Physical Review Applied 16 (2): 024051. https://doi.org/10.1103/PhysRevApplied.16.024051.

Stein, Samuel A., Betis Baheri, Daniel Chen, Ying Mao, Qiang Guan, Ang Li, Bo Fang, and Shuai Xu. 2021. “QuGAN: A Quantum State Fidelity Based Generative Adversarial Network.” In 2021 IEEE International Conference on Quantum Computing and Engineering (QCE), 71–81. https://doi.org/10.1109/QCE52317.2021.00023.


### For the rest of the material follow the link to rolos project https://my.rolos.com/public/project/qml_algs_review